# B - Prueba final módulo 5

#### Introduction

The Dogs vs. Cats dataset that you’ll use was made available by Kaggle as
part of a computer-vision competition in late 2013. You’ll use it to train a
convnet for image classification, getting as much accuracy as possible.

#### Dataset

You can download the original dataset from here:

https://www.kaggle.com/c/dogs-vs-cats/data

This dataset has around 25000 images of dogs and cats (12500 for each
class), tagged by user in captcha tests provided from Asirra.
We propose you to work only with 10% of the data. After downloading and
uncompressing the data file, you’ll create a new dataset containing three
subsets: a training set with 1,000 samples of each class, a validation set
with 500 samples of each class, and a test set with 500 samples of each
class.

### CNN Building
#### Task

1. Explore the data and make the needed preprocessing the image dataset
requires:
a. Read the picture files.
b. Decode the JPEG content to RGB grids of pixels.
c. Convert these into floating-point tensors.
d. Rescale the pixel values (between 0 and 255) to the [0, 1] interval

2. Build a convnet as a stack of alternated Conv2D (with relu activation)
and MaxPooling2D layers. Use the amount of each one that give you the
best accuracy without time and memory over-penalization. End up with
feature maps of size 7 x 7 just before the Flatten layer.

3. Use the loss and optimizer you think better for image classification at
the compiling step.

4. Train the model displaying loss and accuracy curves during it.

5. As you are working with few training samples, overfitting will be
present. Use data augmentation (you can work with
ImageDataGenerator instance in Keras) to avoid it.

6. To further fight overfitting, you’ll also add a Dropout layer to your
model, right before the densely connected classifier

7. Save your model.

8. To go further, we propose you to use a pre trained network: VGG16
model developed by Karen Simonyan and Andrew Zisserman in 2014

### Extra

To go further, we propose you to use a pre trained network: the VGG16
model developed by Karen Simonyan and Andrew Zisserman in 2014.
Try applying feature extraction, importing keras.applications module,
importing VGG16, instantiating the VGG16 convolutional base and using
ImageDataGenerator.


Cargamos las librerías para trabajar con numpy, pandas, matplotlib y keras:

In [ ]:
#cargamos las librerías
import numpy as np
import pandas as pd
import os
import random
import cv2
from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

Importo la libreria zipfile, que sirve para extraer un archivo zip. Una vez que hemos descargado los archivos test1.zip y train.zip de la web de kaggle, en la carpeta /images, procedo a descomprimir su contenido. Tarda un poco:

In [ ]:
import zipfile
with zipfile.ZipFile('./images/test1.zip', 'r') as zip_ref:
    zip_ref.extractall('./images')

with zipfile.ZipFile('./images/train.zip', 'r') as zip_ref:
    zip_ref.extractall('./images')

Cargo con pandas, el csv de los datos de dogs vs. cats:

In [ ]:
#Cargo con pandas, el csv de los datos de dogs vs. cats
data = pd.read_csv("./csv/dogsvscats.csv", header=None)
#muestro los top 10 con las cabeceras
data.head(10)

Clasifico las imágenes de perros y gatos según el nombre del archivo, tanto en la carpeta 'train' como en la carpeta 'test1'. Una vez hecho esto, creo 3 datasets: uno para 'train', con 1000 imágenes de cada categoría; otro para 'valid', con 500 imágenes de cada categoría; finalmente otro para 'test', con 500 imágenes de cada categoría.

In [ ]:
train_dir = './images/train'
test_dir= './images/test1'

train_dogs = ['./images/train/{}'.format(i) for i in os.listdir(train_dir) if 'dog' in i] #coge las imágenes de perros
train_cats = ['./images/train/{}'.format(i) for i in os.listdir(train_dir) if 'cat' in i] #coge las imágenes de gatos

test_dogs = ['./images/test1/{}'.format(i) for i in os.listdir(test_dir) if 'dog' in i] #coge las imágenes de perros
test_cats = ['./images/test1/{}'.format(i) for i in os.listdir(test_dir) if 'cat' in i] #coge las imágenes de gatos

train_img = train_dogs[:1000] + train_cats[:1000] #Slice dataset para training con 1000 datos de cada categoría.
random.shuffle(train_img)
valid_img = train_dogs[:500] + train_cats[:500] # Slice dataset validation con 500 cados de cada categoría
random.shuffle(valid_img)
test_img = test_dogs[:500] + test_cats[:500] #Slice dataset test con 500 casos de cada categoría
random.shuffle(test_img)

Visualizo las 3 primeras imágenes del dataset para 'train'

In [ ]:
for img in train_img[0:3]:
    img = mpimg.imread(img)
    imgplot = plt.imshow(img)
    plt.show()

Redimensiono las imágenes a un tamaño de 150x150 pixels. Inicialmente el tamaño era de 250x250, pero a la hora de crear el feature map, el tamaño resultante era de 13x13, y no de 7x7 como pedía el ejercicio.

In [ ]:
#resize de imagenes
#inicialmente puse tamaño 250*250, pero la última capa terminaba
#con tamaño 13x13 y no 7x7 como pedía el ejercicio
nrows = 150
ncols = 150
channels = 3

X = [] #imágenes
y = [] #etiquetas

for image in train_img:
    X.append(cv2.resize(cv2.imread(image, cv2.IMREAD_COLOR), (nrows, ncols), interpolation=cv2.INTER_CUBIC)) #Lee la imagen
    if 'dog' in image:
        y.append(1)
    else:
        y.append(0)
        
plt.figure(figsize=(20,10))
columns=5

for i in range(5):
    plt.subplot(5/columns+1, columns, i+1)
    imgplot = plt.imshow(X[i])
plt.show()

Convierto las listas en numpy arrays:

In [ ]:
#convert lists in numpy arrays
x = np.array(X)
y = np.array(y)

#print("shape de imágenes de training: ", X.shape)
#print("shape de etiquetas: ",y.shape)


Genero los datos de train y test para el entrenamiento. Muestro el shape de cada una de las variables:

In [ ]:
#split datos en training y validation sets
from sklearn.model_selection import train_test_split

#Asigno el 20% de los datos para validación.
X_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state = 2)

print("shape X_train: ", X_train.shape)
print("shape y_train: ",y_train.shape)
print("shape x_test: ", x_test.shape)
print("shape y_test: ",y_test.shape)

Muestro el nº de elementos de las variables de entrenamiento y test. 1600 elementos para entrenamiento y 400 elementos para test. Declaro una variable, con el valor que usaré para batch, que será 32:

In [ ]:
len_train = len(X_train)
len_test = len(x_test)

print("Nº de elementos de X_train: ",len_train)
print("Nº de elementos de x_test: ",len_test)

#Usaré un batch de 32
batch_size = 32

Creo una red convolucional.

In [ ]:
#preparar la red convolucional
model = models.Sequential()
model.add(layers.Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(150,150,3)))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
# metric = 'accuracy' pq es un problema de clasificación
# loss = 'binary_crossentropy' pq estamos trabajando con 0 y 1
model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=0.00001), metrics=['acc'])

In [ ]:
''' 
uso ImageDataGenerator() para convertir imágenes en tensors. Ésto ayuda a evitar overfitting
Hace las siguientes tareas:
1. Decodifica el contenido jpeg a rgb grids of pixels
2. Convierte éstos en floating-point tensors
3. Reescala los pixel values (entre 0 y 255) al intervalo [0,1]
'''
train_datagen = ImageDataGenerator(rescale=1./255, #Reescala la imagen entre 0 y 1
                               rotation_range=40,
                               width_shift_range=0.2,
                               height_shift_range=0.2,
                               zoom_range=0.2,shear_range= 0.2,
                               horizontal_flip=True,)

validation_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow(X_train, y_train,batch_size=batch_size)
val_generator = validation_datagen.flow(x_test, y_test,batch_size=batch_size)

modelo = model.fit(train_generator,
                   epochs=20,
                   validation_data = val_generator)


Muestro las curvas de accuracy y loss:

In [ ]:
acc = modelo.history['acc']
val_accuracy = modelo.history['val_acc']
loss = modelo.history['loss']
val_loss = modelo.history['val_loss']
epochs = range(1, len(acc)+1)
plt.figure()
plt.plot(epochs, acc,'r-', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'b-', label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Train-Validation-Accuracy-Curve')
plt.ylim([0,1])
plt.legend()
plt.figure()
plt.plot(epochs,loss, 'r-', label='Training Loss')
plt.plot(epochs, val_loss,'b-', label='Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Train-Validation-loss-Curve')
plt.legend()
plt.ylim([0,1])
plt.show()

Vamos a testear nuestro modelo:

In [ ]:
nrows = 150
ncols = 150
channels = 3

X_test = [] #imágenes

for image in train_img[0:10]:
    X_test.append(cv2.resize(cv2.imread(image, cv2.IMREAD_COLOR), (nrows, ncols), interpolation=cv2.INTER_CUBIC)) #Lee la imagen
        
x = np.array(X_test)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
i=0
text_labels = []
plt.figure(figsize=(30,20))
for batch in test_datagen.flow(x, batch_size=1):
    pred = model.predict(batch)
    if pred > 0.5:
        text_labels.append('dog')
    else:
        text_labels.append('cat')
    plt.subplot(5/columns+1, columns, i+1)
    plt.title('This is a '+ text_labels[i])
    imgplot = plt.imshow(batch[0])
    i += 1
    if i % 10 == 0:
        break
plt.show()

Salvo el modelo:

In [ ]:
#save model
model.save_weights('model_weights.h5')
model.save('model_keras.h5')

## VGG16

Genero los valores para train y validation:

In [ ]:
train_generator = train_datagen.flow(X_train, y_train,batch_size=batch_size)
val_generator = validation_datagen.flow(x_test, y_test,batch_size=batch_size)

Importo el modelo VGG16 de keras. Aplico como entrada, imágenes de 150x150. Preentreno el modelo:

In [ ]:
from tensorflow.keras.optimizers import *
from tensorflow.keras.applications import VGG16

# Importamos desde el módulo aaplications al modelo VGG19 ya preentreado.
VGG16_model = VGG16(input_shape=(150, 150, 3), weights=None, include_top=False, input_tensor=None, pooling=None)

model = models.Sequential()

# La parte convolucional de nuestra red será el modelo VGG16
# preentrenado sobre el dataset Imagenet. Esta parte de la red
# tendrá todos sus parámetros congelados.
model.add(VGG16_model)

model.add(layers.Flatten())

model.add(layers.Dense(64,  activation='relu'))
model.add(layers.Dense(32,  activation='relu'))
model.add(layers.Dense(16,  activation='relu'))
model.add(layers.Dense(1,   activation='softmax'))

model.compile(optimizer=Adam(lr=0.00001),
              loss='binary_crossentropy',
              metrics=['acc'])





Presento un resumen del modelo:

In [ ]:
model.summary()

Y ahora sí, entrenamos el modelo:

In [ ]:
model.fit(X_train, y_train, validation_data=(x_test, y_test), batch_size=32, epochs=10)